Below is the computation of w, b, optimal margin, support vectors using CVXOPT and 3rd degree polynomial feature transformation 

In [1]:
import cvxopt
import cvxopt.solvers
import numpy as np

#feature transformation function
#[x1, x2, x3, x4] -> [x1, x2, x3, x4, x1^2, x2^2, x3^2, x4^2, x1^3, x2^3, x3^3, x4^3, x1x2, x1x3, x1x4, x2x3, x2x4, x3x4]
#return the transformed X with new features
def feature_transformation(X): 
    product_X = []
    for i in range(X.shape[0]):
        for j in range(i + 1, X.shape[1]):
            product_X.append(X[:, i] * X[:, j])
    product_X = np.array(product_X).T

    new_X = np.concatenate((X, X**2, X**3, product_X), axis=1)
    return new_X

#quadratic programming solver(CVXOPT) function
#return the optimal w and b
def quad_programming_solver(X, Y): 
    num_of_data, num_of_features = X.shape

    #P matrix 
    P_matrix = np.identity(num_of_features) #identity matrix
    new_row = np.zeros(num_of_features) 
    new_col = np.zeros(num_of_features+1)
    P_matrix = np.vstack((P_matrix, new_row)) #add new col of zeros
    P_matrix = np.hstack((P_matrix, new_col.reshape(num_of_features+1, 1))) #add new row of zeros

    #G matrix 
    Y = Y.reshape(Y.shape[0],1)
    G_matrix = -np.multiply(Y, X) #-yixi
    bcol = np.array([-y for y in Y]) #-yi
    G_matrix = np.hstack((G_matrix, bcol)) #add bcol to G matrix

    P = cvxopt.matrix(P_matrix) #identity with extra row and col of 0's
    q = cvxopt.matrix(np.zeros(num_of_features+1)) #vector of 0's
    G = cvxopt.matrix(G_matrix) #G_matrix with extra row of -yi
    h = cvxopt.matrix(-np.ones(num_of_data)) #vector of -1's

    cvxopt.solvers.options['show_progress'] = False #disable progess to console 
    sol = cvxopt.solvers.qp(P, q, G, h) 
    sol_arr = np.array(sol['x'])
    w = sol_arr[:18]
    b = sol_arr[18]

    return w, b

#optimal margin function 
#return the optimal margin
def find_optimal_margin(w): 
    #optimal margin = 1/||w||
    optimal_margin = 1 / np.linalg.norm(w)
    return optimal_margin

#support vectors function 
#return all support vectors 
def find_support_vectors(X, Y, w, b): 
    #support vectors = data points with yi(wxi+b) = 1 
    support_vectors = []
    for x, y in zip(X, Y):
        result = y * (np.dot(w.reshape(1, w.shape[0]),x) + b)
        if np.round(result, 8) == 1:
            support_vectors.append(x)
    return np.array(support_vectors)


#read file and store in X, Y
file = open("mystery.data", 'r', encoding='utf-8-sig')
dataset = []
for line in file:
    data = line.split(',')
    x_data = [float(x) for x in data[:4]] 
    y_data = int(data[4]) 
    dataset.append((x_data, y_data)) 

X = np.array([x for x, y in dataset])
Y = np.array([y for x, y in dataset])

new_X = feature_transformation(X)
w, b = quad_programming_solver(new_X, Y)
print("w:", w)
print("b:", b)

optimal_margin = find_optimal_margin(w)
print("optimal margin:", optimal_margin)

support_vectors = find_support_vectors(new_X, Y, w, b)
print("support_vectors:", support_vectors)

w: [[ 71.79461668]
 [ 25.64311257]
 [ -2.40163006]
 [  3.47802764]
 [-15.57192139]
 [ 63.80469277]
 [ -9.24984052]
 [-16.4245024 ]
 [  2.94723053]
 [ 32.17361812]
 [ 12.55018643]
 [  8.23461101]
 [-27.25015291]
 [-22.83109875]
 [ 15.59109841]
 [ -1.88268627]
 [ 30.41228841]
 [-55.50412134]]
b: [-0.05630147]
optimal margin: 0.007606385458662345
support_vectors: [[5.46881519e-01 1.42187159e-01 9.27488007e-01 6.37306290e-01
  2.99079396e-01 2.02171882e-02 8.60234003e-01 4.06159307e-01
  1.63560994e-01 2.87462455e-03 7.97856721e-01 2.58847881e-01
  7.77595295e-02 5.07226050e-01 3.48531032e-01 1.31876885e-01
  9.06167708e-02 5.91093941e-01]
 [2.39916154e-01 1.37546595e-01 9.78563885e-01 3.00809091e-01
  5.75597610e-02 1.89190658e-02 9.57587277e-01 9.04861092e-02
  1.38095165e-02 2.60225308e-03 9.37060326e-01 2.72190443e-02
  3.29996501e-02 2.34773284e-01 7.21689602e-02 1.34598130e-01
  4.13752662e-02 2.94360913e-01]
 [3.68484596e-01 4.75546730e-02 5.38746515e-01 6.68085932e-01
  1.35780897e

Below is the eval function
Input: dataset X
Output: predicted y for each datapoint of X

In [ ]:
#feature transformation function
def feature_transformation(X): 
    product_X = []
    for i in range(X.shape[0]):
        for j in range(i + 1, X.shape[1]):
            product_X.append(X[:, i] * X[:, j])
    product_X = np.array(product_X).T

    new_X = np.concatenate((X, X**2, X**3, product_X), axis=1)
    return new_X

#eval function 
def eval(X):
    new_X = feature_transformation(X)

    w = np.array([[71.79461668],
                  [25.64311257],
                  [-2.40163006],
                  [3.47802764],
                  [-15.57192139],
                  [63.80469277],
                  [-9.24984052],
                  [-16.4245024],
                  [2.94723053],
                  [32.17361812],
                  [12.55018643],
                  [8.23461101],
                  [-27.25015291],
                  [-22.83109875],
                  [15.59109841],
                  [-1.88268627],
                  [30.41228841],
                  [-55.50412134]])
    b = [-0.05630147]

    #find Y using wx+b 
    Y = []
    for x in new_X:
        result = np.dot(w.reshape(1, w.shape[0]),x) + b
        if result > 0:
            Y.append(1)
        else:    
            Y.append(-1)
    return np.array(Y)


#prediction of unseen data
predicted_Y = eval(X)
print("Y:\n", predicted_Y.reshape(predicted_Y.shape[0], 1))